## Example: System Identification

In this example we have a typical system identification scenario. We want
to estimate the filter coefficients of an unknown system given by Wo. In     
order to accomplish this task we use an adaptive filter with the same
number of coefficients, N, as the unkown system. The procedure is:         

- Excitate both filters (the unknown and the adaptive) with the signal x. In this case, x is chosen according to the 4-QAM constellation. The variance of x is normalized to 1.
- Generate the desired signal, d = Wo' x + n, which is the output of the unknown system considering some disturbance (noise) in the model. The noise power is given by sigma_n2.
- Choose an adaptive filtering algorithm to govern the rules of coefficient updating.     


##### Adaptive Algorithm used here: LMS     

In [8]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import pydaptivefiltering as pdf

In [2]:
# Aux Functions
def complex_randn(n: int) -> np.array:
    """
    Return a length-`n` vector of iid complex samples X with E|X|^2=1
    """
    return (np.random.randn(n) + np.random.randn(n)*1j) / np.sqrt(2)

## Parameters and Initialization

In [3]:
K = 70                 # Number of iterations
H = np.array([0.32+0.21*1j, -0.3+0.7*1j, 0.5-0.8*1j, 0.2+0.5*1j])
Wo = H                  # Uknown System
sigman2 = 0.04          # Noise Power
N = 4                   # Number of coefficients of the adaptative filter
mu = 0.1                # Convergence factor (step) (0 < μ < 1)

W = np.ones(shape=(N, K+1))
X = np.zeros(N) # Input at a certain iteration (tapped delay line)
x = complex_randn(K)
# complex noise
n = np.sqrt(sigman2/2) * (np.random.randn(K) +
                          np.random.randn(K)*1j)

### desired signal

In [5]:
d = []    

for k in range(K):

    X = np.concatenate(([x[k]], X))[:N]   
    d.append(np.dot(Wo.conj(), X))
    
d = np.array(d) + n   

### Istanciating Adaptive Filter

In [6]:
Filter = pdf.AdaptiveFilter(W[:, 1])

### Adapting with the LMS Algorithm   

In [7]:
Output = pdf.LMS.LMS(Filter, d, x, mu)
# Running the model
Filter, Output, ComplexNoise = LMS_example()

AttributeError: module 'pydaptivefiltering' has no attribute 'LMS'

## Plotting

In [ ]:
plt.figure(figsize=(16, 16)) 

plt.subplot(221)
plt.gca().set_title('Learning Curve for MSE [dB]')
MSE = [abs(err)**2 for err in Output['errors']]
plt.gca().semilogy(MSE)

plt.subplot(222)
plt.gca().set_title('Learning Curve for MSEmin [dB]')
MSEmin = [abs(n)**2 for n in ComplexNoise]
plt.gca().semilogy(MSEmin)

plt.subplot(223)
plt.gca().set_title('Evolution of the Coefficients (Real Part)')
real_part = [coef.real for coef in Output['coefficients']]
plt.gca().plot(real_part)

plt.subplot(224)
plt.gca().set_title('Evolution of the Coefficients (Imaginary Part)')
imag_part = [coef.imag for coef in Output['coefficients']]
plt.gca().plot(imag_part)

plt.tight_layout(pad = 4.0)
plt.grid();

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(d.real[:50], label='desired')
plt.plot(Output["outputs"].real[:50], label='output')
plt.legend();